In [1]:
#imports
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv
from sklearn.preprocessing import MultiLabelBinarizer

load_dotenv()


True

In [2]:
#gets data collected into a csv
def get_csv(region='NA1',game_mode='ARAM',patch='14.13'):
    conn = psycopg2.connect(
        database = os.getenv('DB_NAME'),
        host = os.getenv('DB_HOST'),
        user = os.getenv('DB_USER'),
        password = os.getenv('DB_PASSWORD'),
        port = os.getenv('5432')
    )

    os.makedirs('MatchData', exist_ok=True)
    csv_path = f'MatchData/{region}_{game_mode}_{patch}.csv'

    cursor = conn.cursor()
    patch = patch+'%'

    query_sql = """SELECT * 
    FROM match_data 
    WHERE region = %s 
    AND game_mode = %s 
    AND patch LIKE %s"""

    query = cursor.mogrify(query_sql,(region, game_mode,patch))
    query = query.decode('utf-8')

    try:
        if os.path.exists(csv_path):
            print("Csv found")
        else:
            with open(csv_path,'w') as f:
                cursor.copy_expert("COPY ({}) TO STDOUT WITH CSV HEADER".format(query),f)
            print("Copy to csv successful")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)


In [25]:
#features extraction, encoding and data verification
def get_data(region='NA1',game_mode='ARAM',patch='14.13'):
    csv_path = f'MatchData/{region}_{game_mode}_{patch}.csv'
    if not os.path.exists(csv_path):
        print(f"CSV file not found at {csv_path}")
        return None
    
    df = pd.read_csv(csv_path)[["blue_team","red_team","winning_team"]]

    if df.isnull().any().any():
        df = df.dropna()

    df["blue_team"] = list(df["blue_team"])
    print(df["blue_team"][0][1])
    print(df.head())

    return df

In [26]:
def main(region,game_mode,patch):
    get_csv(region=region,game_mode=game_mode,patch=patch)
    df = get_data(region=region,game_mode=game_mode,patch=patch)

main('NA1','ARAM','14.13')

Csv found
R
                               blue_team                             red_team  \
0  {Rell,Velkoz,Samira,TwistedFate,Bard}    {Tristana,Lux,Neeko,Lulu,Belveth}   
1        {Varus,Lulu,Jhin,Veigar,Viktor}     {Ekko,Maokai,Ashe,KogMaw,RekSai}   
2   {Annie,Darius,Smolder,Hwei,Aphelios}  {Belveth,Quinn,LeeSin,Sylas,Lucian}   
3  {Aphelios,Malphite,Draven,Sett,Janna}   {Bard,Kindred,Swain,Nautilus,Olaf}   
4       {Syndra,Karma,Kalista,Pyke,Sona}    {Garen,Yasuo,TahmKench,Lulu,Nami}   

   winning_team  
0             0  
1             0  
2             0  
3             0  
4             0  
